# Sobre
Este notepad serve para inserir as avaliações colocadas em planilha dentro do banco de dados.

In [14]:
import pandas as pd
import sqlite3

In [12]:
df = pd.read_excel(r'Scraping\Rotulagem de Dados Extraídos.xlsx', 'Avaliação')

In [13]:
df.head()

,ibge,procurement_id,Fonte,Produto/Serviço Extraído,Secretaria Extraída,Avaliação Prod/Serv,Avaliação Secretaria,Observação do Avaliador
0,4119905,27/2023/6,Serviços de Internet com link dedicado para o ...,Serviços de Internet com link dedicado,Câmara Municipal de Ponta Grossa,Correto,Erro de Extração,NaN
1,4119905,26/2023/6,... fornecimento de “PUXADORES PARA JANELAS TI...,Puxadores para janelas tipo “Maxim Ar”,Não informado,Correto,Correto,NaN
2,4119905,25/2023/6,... contratação de empresa habilitada para for...,Relógio protocolador eletrônico,Não informado,Correto,Correto,NaN
3,4119905,24/2023/6,Contratação de empresa especializada para Pres...,Serviços de Saúde e Segurança do Trabalho,Não informado,Correto,Correto,NaN
4,4119905,23/2023/6,... a “aquisição de 08(oito) refrigeradores ti...,Aquisição de 08 refrigeradores tipo frigobar,Não informado,Correto,Correto,NaN


In [16]:
list_id_ibge = list(df['ibge'])
list_id = list(df['procurement_id'])
list_new_value5 = list(df['Avaliação Prod/Serv'])
list_new_value6 = list(df['Avaliação Secretaria'])

In [17]:
def update_procurements_filed(list_id_ibge, list_id, list_new_value, field):

    update = f"UPDATE PROCUREMENTS set {field} = ? where ibge = ? and procurement_id = ?"
    con = sqlite3.connect("scraped.db")
    cursor = con.cursor()
    to_update = []
    for id_ibge, id, new_value in zip(list_id_ibge, list_id, list_new_value):
        to_update.append((new_value, id_ibge, id))
    cursor.executemany(update, to_update)
    con.commit()
    con.close()
    pass

update_procurements_filed(list_id_ibge, list_id, list_new_value5, 'extra5')
update_procurements_filed(list_id_ibge, list_id, list_new_value6, 'extra6')